# Chapter 1 — Introduction to AI Computer Vision (Mechatronics Systems View)
**Course:** Special Topics in Mechatronics Engineering (AI + Computer Vision + Intelligent Controllers)  
**Instructor:** Dr. Mohammad Al Khawaldah  
**Notebook type:** Lecture + Lab (interactive)  
**Last updated:** 2026-02-16

---

## 🎯 Learning Outcomes
By the end of Chapter 1, you will be able to:

1. Explain the full **Vision→Decision→Controller** pipeline (system engineering, not model-only).
2. Define core terms precisely: **model, training, inference, dataset, labels, classes, confidence, threshold, NMS, tracking, domain shift, latency**.
3. Run a complete example application (frame → inference → decision gate → “actuator command” + evidence logging).
4. Ask the right **engineering questions** and identify common **failure modes** in industrial vision systems.

> In mechatronics, the model is only a part of the system. The rest is engineering.

---
# 1) Introduction: AI Computer Vision in Mechatronics

Computer Vision transforms raw pixels into structured understanding (objects, states, defects, motion).  
Mechatronics turns that understanding into **actions** that affect the physical world.

### Why it is different from “pure AI”
A real mechatronics vision system must satisfy:
- **Latency constraints** (real-time control, moving targets, conveyor speed)
- **Safety constraints** (false alarms create “alarm fatigue”; missed detections can cause accidents)
- **Hardware constraints** (CPU/GPU, memory, power, edge vs cloud)
- **Environmental variation** (lighting, dust, glare, camera shift) → domain shift

We do not just train models — we design **end-to-end systems**.

---
# 2) The Complete AI Vision Pipeline (End-to-End System)

Canonical pipeline in industrial mechatronics:

**Sensor → Preprocessing → Model Inference → Post-processing → Decision Logic → Controller Interface → Actuator → Logging**

You must be able to explain what happens at each step, and what can go wrong.

In [ ]:
# =========================================================
# 2.1 Flowchart diagram (auto-generated) using Graphviz
# =========================================================
!pip -q install graphviz

from graphviz import Digraph

g = Digraph("vision_pipeline", format="png")
g.attr(rankdir="LR", fontsize="12")

g.node("S", "Sensor\n(Camera/Stream)", shape="box")
g.node("P", "Preprocessing\n(resize, normalize, ROI)", shape="box")
g.node("M", "Model Inference\n(CLS/DET/SEG)", shape="box")
g.node("PP", "Post-processing\n(threshold, NMS, tracking)", shape="box")
g.node("D", "Decision Logic\n(state machine, safety gate)", shape="box")
g.node("C", "Controller Interface\n(serial/MQTT/REST/ROS)", shape="box")
g.node("A", "Actuator\n(LED/buzzer/gate/motor)", shape="box")
g.node("L", "Logging\n(evidence + metrics)", shape="box")

g.edges([("S","P"), ("P","M"), ("M","PP"), ("PP","D"), ("D","C"), ("C","A"), ("D","L")])

g

### Engineering interpretation
- Preprocessing changes both accuracy and latency.
- Post-processing prevents duplicated boxes (NMS) and stabilizes decisions (temporal smoothing).
- Decision logic is where safety policies live (e.g., “3 consecutive frames required”).
- Logging is mandatory for audits and debugging.

---
# 3) Industrial Applications (Detailed)

## A) Truck Load Compliance Inspection
**Goal:** Detect whether a truck bed is covered/uncovered/irregular and optionally classify visible load type.

**Inputs:** fixed CCTV stream (RTSP). Sometimes a separate ANPR camera provides plate number as API.  
**Outputs:** violation/no-violation event + evidence snapshot + optional actuator (gate/buzzer).

**System steps:**
1. Detect truck/bed region (DET) → crop ROI for better accuracy.
2. Classify cover state (CLS) OR segment cover/exposed regions (SEG).
3. Temporal gate: stable violation for N frames to avoid “one-frame noise”.
4. Merge repeated detections into one event per truck using tracking ID.
5. Log timestamp, confidence, snapshot, and plate ID (fusion).

**Hard real-world conditions:**
glare, shadows, night IR, motion blur, camera shift (domain shift), multiple trucks, occlusions.

## B) PPE Safety (Helmet detection)
Detect person + helmet; alarm if person without helmet persists >2 seconds (avoid alarm spam).

## C) Conveyor Quality Inspection
Detect defects/missing parts and trigger reject actuator (air jet/robot arm). Strict latency.

## D) Drone Vision
Detect targets from moving camera. Bandwidth + motion blur constraints.

---
# 4) Definitions (Core Terms)

### Model
A model is a function **fθ(x)** mapping input image x → output y. θ are learned parameters.

### Training vs Inference
- Training: learn θ from labeled dataset by minimizing a loss.
- Inference: run trained model on new frames → predictions.

### Dataset, Labels, Classes
- Dataset: samples (images/frames) + labels.
- Label/Annotation: ground truth (class / boxes / masks).
- Class: category name (“covered”, “uncovered”, “helmet”, …).

### Confidence & Threshold
- Confidence: model score for prediction (not always calibrated probability).
- Threshold: accept prediction only if score ≥ threshold.

### NMS
Removes duplicated overlapping boxes (detection).

### Tracking
Persistent object IDs across frames; prevents multiple events per same object.

### Overfitting
Train ↑, val ↓. Model memorizes.

### Domain shift
Training distribution ≠ deployment distribution (angle/lighting/weather). #1 industrial failure.

### Latency
Capture → decision time. Must meet real-time needs.

---
# 5) Complete Example Application (End-to-End)
## “Uncovered Truck” → Event + Buzzer Command + Evidence

We will run a full industrial-style loop:
1. Read frames (here: images downloaded automatically).
2. Simulate model inference (toy classifier for teaching pipeline).
3. Apply threshold and temporal gate (N consecutive detections).
4. Trigger “actuator command” (print command).
5. Log evidence (save snapshot + metadata).

Later chapters replace toy model with YOLO/real models.

## 5.1 Auto-download example images (no manual copy/paste)
This cell downloads images automatically into the runtime.

In [ ]:
# =========================================================
# 5.1 Download example images automatically (public-domain sources)
# =========================================================
import os, requests

os.makedirs("chapter1_images", exist_ok=True)

image_urls = {
  "dump_truck": "https://upload.wikimedia.org/wikipedia/commons/3/3f/Dump_truck.jpg",
  "construction_waste": "https://upload.wikimedia.org/wikipedia/commons/1/14/Construction_and_demolition_waste.jpg",
  "safety_helmet": "https://upload.wikimedia.org/wikipedia/commons/5/5d/Hard_hat.jpg"
}

for name, url in image_urls.items():
    out = f"chapter1_images/{name}.jpg"
    if not os.path.exists(out):
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        with open(out, "wb") as f:
            f.write(r.content)
        print("Downloaded:", out)
    else:
        print("Exists:", out)

In [ ]:
# =========================================================
# 5.2 Visualize images
# =========================================================
import cv2
import matplotlib.pyplot as plt

def show(path, title=None):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(7,4))
    plt.imshow(img)
    plt.axis("off")
    plt.title(title or os.path.basename(path))
    plt.show()

for p in sorted(os.listdir("chapter1_images")):
    show(os.path.join("chapter1_images", p), title=p)

## 5.3 A toy “model” (teaching device)
We compute a simple texture score using Laplacian variance and map it to [0,1].
This illustrates: model input → score → threshold decision.

In [ ]:
import numpy as np

def toy_uncovered_score(rgb_img: np.ndarray) -> float:
    gray = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    texture = lap.var()
    score = 1 / (1 + np.exp(-(texture - 200) / 80))
    return float(score)

def predict_score(image_path: str) -> float:
    rgb = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    return toy_uncovered_score(rgb)

## 5.4 Temporal gate (safety rule)
Never trigger from a single frame. Require N consecutive frames above threshold + cooldown.

In [ ]:
import time
from dataclasses import dataclass

@dataclass
class TriggerPolicy:
    threshold: float = 0.55
    min_consecutive: int = 2
    cooldown_s: float = 1.0

class TemporalGate:
    def __init__(self, policy: TriggerPolicy):
        self.p = policy
        self.consec = 0
        self.last_fire = 0.0

    def update(self, score: float) -> bool:
        now = time.time()
        if now - self.last_fire < self.p.cooldown_s:
            self.consec = 0
            return False

        if score >= self.p.threshold:
            self.consec += 1
        else:
            self.consec = 0

        if self.consec >= self.p.min_consecutive:
            self.last_fire = now
            self.consec = 0
            return True
        return False

## 5.5 Controller command + evidence logging
We simulate Arduino/PLC command and save evidence snapshot + JSON metadata.

In [ ]:
import json, shutil
from datetime import datetime

os.makedirs("evidence", exist_ok=True)

def send_controller_command(cmd: str):
    print(f"[CONTROLLER] {cmd}")

def log_evidence(image_path: str, score: float, reason: str):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_img = f"evidence/{ts}_{os.path.basename(image_path)}"
    shutil.copy(image_path, out_img)
    meta = {"timestamp": ts, "image": out_img, "score": score, "reason": reason}
    with open(out_img + ".json", "w") as f:
        json.dump(meta, f, indent=2)
    print("[LOGGED]", meta)

## 5.6 Run the end-to-end demo loop
We loop over “frames” (images) and apply the system logic.

In [ ]:
policy = TriggerPolicy(threshold=0.55, min_consecutive=2, cooldown_s=1.0)
gate = TemporalGate(policy)

frame_paths = [
    "chapter1_images/dump_truck.jpg",
    "chapter1_images/construction_waste.jpg",
    "chapter1_images/dump_truck.jpg",
    "chapter1_images/construction_waste.jpg",
]

print("Policy:", policy)

for i, fp in enumerate(frame_paths):
    score = predict_score(fp)
    fire = gate.update(score)
    print(f"Frame {i:02d}: {os.path.basename(fp):25s} score={score:.3f} fire={fire}")
    if fire:
        send_controller_command("BUZZER_ON")
        log_evidence(fp, score, reason=f"score>=threshold for {policy.min_consecutive} consecutive frames")

✅ You executed a full industrial-style pipeline:
frames → model score → threshold + temporal smoothing → actuator command → evidence logging.

Later chapters will replace the toy model with YOLO models trained on your datasets.

---
# 6) Key Questions (Engineering Checklist)

### System questions
1. Edge vs cloud inference? What if network fails?
2. End-to-end latency budget? Required FPS?
3. Cost of false positives vs false negatives?
4. Tracking needed to enforce “one event per object”?
5. What evidence must be stored (snapshot + confidence + reason)?

### Data questions
1. Does dataset match deployment camera angle/lighting?
2. Is dataset balanced?
3. Are labels consistent?
4. Do we have rare critical cases (night/rain/occlusion)?

### Model questions
1. Need classification/detection/segmentation?
2. Smallest model meeting accuracy+speed?
3. Which threshold matches safety policy?

---
# 7) Failure Modes (Real-World)

### Data failures
Label noise, dataset bias (day-only), leakage, missing corner cases.

### Model failures
Overfitting, slow model (latency), poor confidence calibration.

### Deployment failures
Camera moved, dirty lens, compression artifacts, new unseen objects (domain drift).

### System logic failures
No temporal smoothing, no cooldown, no tracking, no logging.

---
# 8) Exercises (Graded)

## Exercise 1 — Threshold trade-off
Change `policy.threshold` to 0.80 and rerun. Explain what happens (FP vs FN).

## Exercise 2 — Stronger temporal gating
Set `min_consecutive` to 3. Add more frames to `frame_paths`. Observe stability.

## Exercise 3 — Evidence
Open the `evidence/` folder and inspect saved files. Why is this mandatory in compliance systems?

## Exercise 4 — Design (written)
Pick one application (truck/PPE/conveyor/drone) and write:
- pipeline blocks
- where domain shift can happen
- what you will log as evidence

---
# ✅ Chapter 1 Summary
You learned:
- AI vision is a **system**, not only a model.
- Key terms and how they connect in the pipeline.
- A complete end-to-end demo with safety gating and evidence logging.
- Engineering questions + failure modes.

👉 Next: Module 2 — Image Classification (dataset, training, evaluation, deployment).